In [1]:
import tkinter as tk
from tkinter import *
import pygame
from tkinter import filedialog 
from tkinter import simpledialog
from tkinter import messagebox
import pandas as pd
import time
from mutagen.mp3 import MP3
import tkinter.ttk as ttk
import csv

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
root = Tk()
root.title('Music Player')
root.iconbitmap()
root.geometry("550x460")

''

In [3]:
def add_song():
    song = filedialog.askopenfilename(initialdir=r"C:/IP project/Songs/",
                                      title='Choose a song',
                                      filetypes=(("mp3 files","*.mp3"),))
    directory = song
    # to remove directory info and mp3 extention from o/p
    song = song.replace("C:/IP project/Songs/","")
    song = song.replace(".mp3","")
    # add song to list box
    song_b.insert(END,song)
    #print(song_b.size())

In [4]:
# play selected song
def play():
    # set stopped variable to false so song can play
    global stopped
    stopped = False
    song = song_b.get(ACTIVE)
    song = fr"C:/IP project/Songs/{song}.mp3"
    
    pygame.mixer.music.load(song)
    pygame.mixer.music.play(loops=0)
    # to show time
    play_time()
    
    # update slider to position
    slider_position = int(song_length)
    my__slider.config(to=slider_position, value=0)
    
    # get current volume
    #curr_vol = pygame.mixer.music.get_volume()
    #slider_label.config(text=curr_vol*100)

In [5]:
# stop playing current song
global stopped
stopped = False
def stop():
    #reset slider and status bar
    status_bar.config(text='')
    my__slider.config(value = 0)
    #stop song from playing
    pygame.mixer.music.stop()
    song_b.selection_clear(ACTIVE)
    
    #to clear statusbar
    status_bar.config(text='')
    
    #set stop variable to true
    global stopped 
    stopped = True

In [6]:
# pause and unpause the current song
global paused
paused = False
def pause(is_paused):
    global paused
    paused = is_paused
    if paused:
        pygame.mixer.music.unpause() # for unpause
        paused = False
    else:
        pygame.mixer.music.pause() # for pause
        my__slider.config(state = tk.DISABLED) # if pause button is hit the slider gets disabled
        paused = True

In [7]:
# to play next song forward button
def next_song():
    # reset slider and status bar
    status_bar.config(text='')
    my__slider.config(value = 0)
    
    # to get the current song's NUMBER IN A TUPLE
    next_one = song_b.curselection()
    # for next song just add 1
    next_one = next_one[0]+1
    # to get next song title 
    song = song_b.get(next_one)
    
    song = fr"C:/IP project/Songs/{song}.mp3"
    pygame.mixer.music.load(song)
    pygame.mixer.music.play(loops=0)
    
    # to move active bar
    song_b.selection_clear(0, END)
    
    # new song active bar
    song_b.activate(next_one)
    
    # the selected bg
    song_b.selection_set(next_one, last = None)

In [8]:
# back button
def prev_song():
    # reset slider and status bar
    status_bar.config(text='')
    my__slider.config(value = 0)
    
    # to get the current song's no in a tuple
    prev_one = song_b.curselection()
    # for next song just subtract 1
    prev_one = prev_one[0]-1
    # to get next song title 
    song = song_b.get(prev_one)
    
    song = fr"C:/IP project/Songs/{song}.mp3"
    pygame.mixer.music.load(song)
    pygame.mixer.music.play(loops=0)
    
    # to move active bar
    song_b.selection_clear(0, END)
    
    # new song active bar
    song_b.activate(prev_one)
    
    # the selected bg
    song_b.selection_set(prev_one, last = None)

In [9]:
# to add many songs to the playlist
def add_many_songs():
    songs = filedialog.askopenfilenames(initialdir=r"C:/IP project/Songs/",
                                        title='Choose songs',
                                        filetypes=(("mp3 files","*.mp3"),))
    #loop through song list and replace directories
    for song in songs:
        song = song.replace("C:/IP project/Songs/","")
        song = song.replace(".mp3","")
        #add song to list box
        song_b.insert(END, song)

In [10]:
# to delete A song
def delete_song():
    stop()
    song_b.delete(ANCHOR)   # currently selected one
    pygame.mixer.music.stop()

In [11]:
# apparently music still plays after closing the gui to stop that
def on_closing():
    if (tk.messagebox.askyesno(title="Save songs as playlist", 
                               message="Do you want to save songs as a Playlist")):
        save()
    if (tk.messagebox.askyesno(title="Closing", 
                               message="Do you really want to exit?")):
        stop_music()
        root.destroy()

In [12]:
def stop_music():
    pygame.mixer.music.stop()

In [13]:
# to delete all songs
def delete_all_songs():
    stop()
    song_b.delete(0, END)
    pygame.mixer.music.stop() # stop music if playing

In [14]:
# grab song length
def play_time():
    if stopped:
        # check for double timing
        return # breaks out of a function
    
    # get current song elapsed time
    current_time = pygame.mixer.music.get_pos() / 1000
    
    # throw up temp label to get data
    # slider_label.config(text=f'Slider: {int(my__slider.get())} and Song Pos: {int(current_time)}')

    # convert to time format
    conv_curr_time = time.strftime('%M:%S', time.gmtime(current_time))
    
    # for knowing currently playing song
    current_song = song_b.curselection()  # returns a number
    # to get next song title 
    song = song_b.get(ACTIVE)
    # back into directory structure
    song = fr"C:/IP project/Songs/{song}.mp3"

    # get song length with mutagen
    song_mut = MP3(song)
    
    # get song length
    global song_length
    global paused
    song_length = song_mut.info.length  # in secs
    
    # convert to time format
    conv_song_length = time.strftime('%M:%S', time.gmtime(song_length))
    if (paused == True):
        my__slider.config(state = tk.DISABLED)    
    else:
        my__slider.config(state = tk.NORMAL)
        if int(my__slider.get()) == int(song_length):
            status_bar.config(text=f'Time elapsed: {conv_song_length}  of  {conv_song_length}   ')
            try:
                next_song()   
            except:
                messagebox.showinfo("Playlist ended", "Your Playlist has ended")

        elif int(my__slider.get() == int(current_time)):
            # slider hasn't been moved
            slider_position = int(song_length)
            my__slider.config(to = slider_position, value = int(current_time))
            status_bar.config(text = f'Time elapsed: {conv_curr_time}  of  {conv_song_length}   ')

        else:
            # slider has been moved
            # to update it!
            slider_position = int(song_length) 
            my__slider.config(to = slider_position, value = int(my__slider.get()))
            # convert to time format
            conv_curr_time = time.strftime('%M:%S', time.gmtime(int(my__slider.get())))
            # o/p time to statusbar 
            status_bar.config(text=f'Time elapsed: {conv_curr_time}  of  {conv_song_length}   ')

            # move the thing along by a second
            next_time = int(my__slider.get()) + 1
            my__slider.config(value = next_time)
        
            
    # update time
    status_bar.after(1000, play_time)

In [15]:
# create slider function
def slide(x):
    #slider_label.config(text=f'{int(my__slider.get())} of {int(song_length)}')
    song = song_b.get(ACTIVE)
    song = fr"C:/IP project/Songs/{song}.mp3"
    
    pygame.mixer.music.load(song)
    pygame.mixer.music.play(loops=0, start=int(my__slider.get()))

In [16]:
# create volume function
def volume(x):
    pygame.mixer.music.set_volume(vol_slider.get())  # set values from 0 to 1(floats) so is the slider value 
    
    # get current volume
    #curr_vol = pygame.mixer.music.get_volume()
    #slider_label.config(text=curr_vol)

In [17]:
def open_():
    df=pd.DataFrame([])
    csvv = filedialog.askopenfilename(initialdir=r"C:/IP project/Playlists/",
                                      title="Choose a playlist",
                                      filetypes=(("csv files","*.csv"),))
    df = pd.read_csv(csvv, header = None, skiprows=1)

    for song in df[0]:
        song_name = song
        #add song to list box
        song_b.insert(END, song)

In [18]:
def save():
    if song_b.size()!=0:
        playlist = simpledialog.askstring("Name", "Name of the playlist")
        data={}
        l1=[]
        l2=[]

        for x in song_b.get(0,END):
            l1.append(x)
            l2.append(fr"C:/IP project/Songs/{x}.mp3")

        data = {'Name':l1,'Dir':l2}    
        df4 = pd.DataFrame(data)
        df4.to_csv(f"C:/IP project/Playlists/{playlist}.csv",index=False) 
        
    else:
        messagebox.showinfo("Empty!","Your Playlist is empty")   

In [19]:
# initailise pygame mixer
pygame.mixer.init()

In [20]:
# create master frame
master_frame = Frame(root)
master_frame.pack(pady=20)

# create a playlist box
song_b = Listbox(master_frame,
                 bg='black',
                 fg='yellow',
                 width=60,
                 selectbackground="purple",
                 selectforeground="gold")
song_b.grid(row=0, column=0)

# create volume label frame
volume_frame = LabelFrame(master_frame, text='Volume')
volume_frame.grid(row=3, column=0) 

# create player control frame
controls_frame = Frame(master_frame)
controls_frame.grid(row=1,column=0,pady=20,padx=10)

In [21]:
# define player control buttons with images
back_btn_img = tk.PhotoImage(file=r"C:/IP project/Images/Previous.gif")
forward_btn_img = tk.PhotoImage(file=r"C:/IP project/Images/Next.gif")
play_btn_img = tk.PhotoImage(file=r"C:/IP project/Images/play.gif")
pause_btn_img = tk.PhotoImage(file=r"C:/IP project/Images/Pause.gif")
stop_btn_img = tk.PhotoImage(file=r"C:/IP project/Images/Stop.gif")

In [22]:
# creating buttons
back_button = Button(controls_frame, 
                     image=back_btn_img, 
                     borderwidth=0,
                     command=prev_song)
forward_button = Button(controls_frame, 
                        image=forward_btn_img, 
                        borderwidth=0,
                        command=next_song)
play_button = Button(controls_frame,
                     image=play_btn_img, 
                     borderwidth=0,
                     command=play)
stop_button = Button(controls_frame, 
                     image=stop_btn_img, 
                     borderwidth=0,
                     command=stop)
pause_button = Button(controls_frame,
                      image=pause_btn_img, 
                      borderwidth=0,
                      command=lambda: pause(paused))

In [23]:
# allignment of buttons
back_button.grid(row=0,column=0,padx=5)
forward_button.grid(row=0,column=4,padx=5)
play_button.grid(row=0,column=2,padx=5)
pause_button.grid(row=0,column=1,padx=5)
stop_button.grid(row=0,column=3,padx=5)

In [24]:
# create a menu
my_menu = Menu(root)
root.config(menu=my_menu)

In [25]:
# add song menu
add_song_menu = Menu(my_menu)
my_menu.add_cascade(label="Add songs", menu=add_song_menu)
add_song_menu.add_command(label="Add one song to playlist", command=add_song)
# add many songs to the playlist
add_song_menu.add_command(label="Add many songs to playlist", command=add_many_songs)

# create delete song menu
remove_song_menu = Menu(my_menu)
my_menu.add_cascade(label='Remove Songs', menu=remove_song_menu)
remove_song_menu.add_cascade(label="Delete a song from playlist", command=delete_song)
remove_song_menu.add_cascade(label="Delete All songs from playlist", command=delete_all_songs)

# create open playlist
my_playlist = Menu(my_menu)
my_menu.add_cascade(label="Playlists", menu=my_playlist)
my_playlist.add_command(label="Open playlists", command=open_)
my_playlist.add_command(label="Save playlist", command=save)

In [26]:
# create statusbar
status_bar = Label(root, text='', bd=1,relief=GROOVE, anchor=E)
status_bar.pack(fill=X, side=BOTTOM, ipady=2)

In [27]:
# creating the slider in the menu box
my__slider = ttk.Scale(master_frame, 
                       from_=0, to=100, 
                       orient=HORIZONTAL, 
                       value=0, 
                       command=slide, 
                       length=360)
my__slider.grid(row=2,column=0,pady=10)

In [28]:
# create temporary slider label for testing purpose
# slider_label = Label(root, text="0")
# slider_label.pack(pady=10)

In [29]:
# create volume control slider
vol_slider = ttk.Scale(volume_frame, 
                       from_=0, 
                       to=1, 
                       orient=HORIZONTAL, 
                       value=1, 
                       command=volume, 
                       length=125)
vol_slider.pack(pady=10)

In [30]:
root.protocol("WM_DELETE_WINDOW",on_closing)

''

In [ ]:
root.mainloop()